In [ ]:
# ✅ Install dependencies if needed
# !pip install tensorflow matplotlib numpy --quiet

import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
import numpy as np
import matplotlib.pyplot as plt

# -------------------------------
# STEP 1: LOAD AND PREPROCESS MNIST
# -------------------------------
(x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Flatten images (28x28 -> 784)
x_train = x_train.reshape((len(x_train), 784))
x_test = x_test.reshape((len(x_test), 784))

# -------------------------------
# STEP 2: DEFINE AUTOENCODER ARCHITECTURE
# -------------------------------
encoding_dim = 64  # Size of the latent space

# Encoder with sparsity constraint (L1 regularization)
input_img = layers.Input(shape=(784,))
encoded = layers.Dense(encoding_dim, activation='relu',
                       activity_regularizer=regularizers.l1(1e-4))(input_img)
decoded = layers.Dense(784, activation='sigmoid')(encoded)

# Build Autoencoder
autoencoder = models.Model(input_img, decoded)

# Separate encoder model (for visualization)
encoder = models.Model(input_img, encoded)

# -------------------------------
# STEP 3: COMPILE MODEL
# -------------------------------
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# -------------------------------
# STEP 4: TRAIN AUTOENCODER
# -------------------------------
history = autoencoder.fit(
    x_train, x_train,
    epochs=1000,
    batch_size=256,
    shuffle=True,
    validation_data=(x_test, x_test)
)

# -------------------------------
# STEP 5: EVALUATE MODEL
# -------------------------------
encoded_imgs = encoder.predict(x_test)
decoded_imgs = autoencoder.predict(x_test)

# -------------------------------
# STEP 6: VISUALIZE RECONSTRUCTION
# -------------------------------
n = 10  # number of digits to display
plt.figure(figsize=(20, 4))
for i in range(n):
    # Original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28), cmap='gray')
    plt.title("Original")
    plt.axis('off')

    # Reconstructed
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28), cmap='gray')
    plt.title("Reconstructed")
    plt.axis('off')
plt.show()

# -------------------------------
# STEP 7: CHECK SPARSITY LEVEL
# -------------------------------
sparsity = np.mean(np.abs(encoded_imgs) < 0.05)
print(f"\nSparsity (fraction of near-zero activations): {sparsity*100:.2f}%")


Epoch 1/1000
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.6567 - val_loss: 0.6155
Epoch 2/1000
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.5832 - val_loss: 0.5535
Epoch 3/1000
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 0.5275 - val_loss: 0.5038
Epoch 4/1000
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.4827 - val_loss: 0.4638
Epoch 5/1000
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.4466 - val_loss: 0.4314
Epoch 6/1000
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.4174 - val_loss: 0.4050
Epoch 7/1000
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.3935 - val_loss: 0.3834
Epoch 8/1000
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.3739 - val_loss: 0.3656
Epoch 9/1000
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.3577 - val_loss: 0.3508
Epoch 10/1000
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.3442 - val_loss: 0.3385
Epoch 11/1000
235/235 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.3329 - val_loss: 0.3281
Epoch 12/1000
235/235 ━━━━